In [ ]:
from typing import Optional, Dict
from fmbase.source.merra2.model import YearMonth, load_batch
from fmgraphcast.config import hydra_config_files
import hydra
import xarray as xa
import  time
from fmbase.util.config import configure, cfg
from fmbase.plot.image import plot

In [ ]:
hydra.initialize( version_base=None, config_path="../config" )
configure(  'merra2-test' )
t0 = time.time()

pvar = "temperature"
res,levels,steps = cfg().model.res,  cfg().task.levels,  cfg().model.steps
year, month, day =  cfg().task.year,  cfg().task.month,  cfg().task.day
train_steps, eval_steps = cfg().task.train_steps, cfg().task.eval_steps
(model_config,task_config) = hydra_config_files()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Load MERRA2 Data
#-----------------

params: Dict[str,Dict] = None
state = {}
dts         = cfg().task.data_timestep
start = YearMonth(year,month)
end = YearMonth(year,month+1)
target_lead_times = [ f"{iS*dts}h" for iS in range(1,train_steps+1) ]
eval_lead_times =   [ f"{iS*dts}h" for iS in range(1,eval_steps+1) ]

In [ ]:
print( "  --------------------- MERRA2 ---------------------")
example_batch: xa.Dataset = load_batch( year, month, day, ndays, cfg().task, vars=[pvar] )

print("Loaded Batch:")
for vname, dvar in example_batch.data_vars.items():
	print( f" {vname}{list(dvar.dims)}: shape={dvar.shape}")

In [ ]:
plot( example_batch, pvar )